### bulk ExpressionSet

In [1]:
library(SummarizedExperiment)

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles, rowRanges, rowRanks,
    rowSdDiffs, rowSds, rowSums2, ro

In [2]:
Matrix <- read.table("/lustre/scratch117/cellgen/team205/rl20/CTCL/deconvolution/GSE121212_readcount_rmdup.txt", 
                     check.names=F) ### count matrix

In [3]:
head(Matrix);dim(Matrix)

,AD_004_lesional,AD_004_non-lesional,AD_005_lesional,AD_005_non-lesional,AD_006_lesional,AD_006_non-lesional,AD_007_lesional,AD_007_non-lesional,AD_009_lesional,AD_009_non-lesional,⋯,AD_033_chronic_lesion,AD_034_non-lesional,AD_034_chronic_lesion,AD_032_chronic_lesion,AD_035_non-lesional,AD_035_chronic_lesion,AD_036_non-lesional,AD_036_chronic_lesion,AD_037_non-lesional,AD_037_chronic_lesion
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
5S_rRNA,0,3,9,12,2,3,6,4,2,5,⋯,1,2,3,3,3,4,3,5,6,2
7SK,9,6,25,25,7,8,9,8,6,15,⋯,3,8,2,5,12,5,7,5,8,3
A1BG,2,0,4,3,1,1,0,0,4,0,⋯,1,1,2,0,1,2,0,0,1,1
A1BG-AS1,5,0,11,8,7,8,13,2,6,19,⋯,4,0,7,7,4,4,10,13,8,4
A1CF,2,3,2,8,1,2,0,5,1,5,⋯,3,0,1,0,0,2,0,0,1,0
A2M,308,353,2544,1366,978,406,2761,928,2796,1891,⋯,1463,1391,1714,1998,683,1154,2361,2374,1040,2755


[1] 31362   147

In [4]:
is_genename_uniq <- !duplicated(rownames(Matrix))
Matrix <- Matrix[is_genename_uniq, ]
dim(Matrix)

[1] 31362   147

In [5]:
tt <- apply(Matrix, 2, sum)
del <- which(tt==0)
del

named integer(0)

In [6]:
fdat_df <- data.frame(gene_name=rownames(Matrix), stringsAsFactors=F)
rownames(fdat_df) <- rownames(Matrix)
metadata_fdat_df <- data.frame(labelDescription= c("gene name"), row.names=c("gene_name"))
head(fdat_df)

,gene_name
,<chr>
5S_rRNA,5S_rRNA
7SK,7SK
A1BG,A1BG
A1BG-AS1,A1BG-AS1
A1CF,A1CF
A2M,A2M


In [7]:
metadata <- read.table("/lustre/scratch117/cellgen/team205/rl20/CTCL/deconvolution/GSE121212_meta.xls", 
                       check.names=F, sep="\t", header = T)
head(metadata)

,sample_type,condition
,<chr>,<chr>
AD_004_lesional,AD_L,AD
AD_004_non-lesional,AD_NL,AD
AD_005_lesional,AD_L,AD
AD_005_non-lesional,AD_NL,AD
AD_006_lesional,AD_L,AD
AD_006_non-lesional,AD_NL,AD


In [8]:
pdat_df <- data.frame(sample_type = metadata$sample_type, 
                      condition = metadata$condition,
                      stringsAsFactors = F)
rownames(pdat_df) <- rownames(metadata)
head(pdat_df)

,sample_type,condition
,<chr>,<chr>
AD_004_lesional,AD_L,AD
AD_004_non-lesional,AD_NL,AD
AD_005_lesional,AD_L,AD
AD_005_non-lesional,AD_NL,AD
AD_006_lesional,AD_L,AD
AD_006_non-lesional,AD_NL,AD


In [9]:
metadata_pdat_df <- data.frame(labelDescription= c("lesion_nonlesion", "Condition"), 
                               row.names=c("sample_type", "condition"))

In [10]:
metadata_pdat_df

,labelDescription
,<chr>
sample_type,lesion_nonlesion
condition,Condition


In [11]:
Matrix <- Matrix[ ,rownames(pdat_df)] ### the order of rownames(pdat_df) and colnames(Matrix) might be different

In [12]:
all.equal(colnames(Matrix), rownames(pdat_df))

[1] TRUE

In [13]:
bulk_eset <- ExpressionSet(
  assayData = data.matrix(Matrix),
  phenoData=new("AnnotatedDataFrame", data = pdat_df, varMetadata = metadata_pdat_df),
  featureData=new("AnnotatedDataFrame", data = fdat_df, varMetadata = metadata_fdat_df))

In [14]:
bulk_eset

ExpressionSet (storageMode: lockedEnvironment)
assayData: 31362 features, 147 samples 
  element names: exprs 
protocolData: none
phenoData
  sampleNames: AD_004_lesional AD_004_non-lesional ...
    AD_037_chronic_lesion (147 total)
  varLabels: sample_type condition
  varMetadata: labelDescription
featureData
  featureNames: 5S_rRNA 7SK ... snoZ5 (31362 total)
  fvarLabels: gene_name
  fvarMetadata: labelDescription
experimentData: use 'experimentData(object)'
Annotation:  

In [17]:
saveRDS(bulk_eset, file="/lustre/scratch117/cellgen/team205/rl20/CTCL/deconvolution/GSE121212_readcount_ExpressionSet.rds")

### Single cell ExpressionSet

In [15]:
library(sceasy)
library(reticulate)

Loading required package: reticulate



In [20]:
h5ad_file <- "/lustre/scratch117/cellgen/team205/rl20/CTCL/object-new/CTCL1-8_raw_downsample0.2_anno_for_deconv.h5ad"
sceasy::convertFormat(h5ad_file, from="anndata", to="seurat",
                       outFile='/lustre/scratch117/cellgen/team205/rl20/CTCL/object-new/CTCL1-8_raw_downsample0.2_anno_for_deconv.rds')

X -> counts



An object of class Seurat 
33538 features across 54611 samples within 1 assay 
Active assay: RNA (33538 features, 0 variable features)

In [16]:
library(Seurat)
library(BisqueRNA)
library(Biobase)
library(Matrix)

Attaching SeuratObject


Attaching package: ‘Seurat’


The following object is masked from ‘package:SummarizedExperiment’:

    Assays



Attaching package: ‘Matrix’


The following object is masked from ‘package:S4Vectors’:

    expand




In [17]:
seurat_object <- readRDS("/lustre/scratch117/cellgen/team205/rl20/CTCL/object-new/CTCL1-8_raw_downsample0.2_anno_for_deconv.rds")
seurat_object

An object of class Seurat 
33538 features across 55815 samples within 1 assay 
Active assay: RNA (33538 features, 0 variable features)

In [18]:
gene_exprs.matrix <- seurat_object@assays$RNA@counts
gene_exprs.matrix[1:20,1:20]


   [[ suppressing 20 column names ‘AAACCTGAGAATGTTG-1’, ‘AAACCTGAGTTCGCAT-1’, ‘AAACCTGCATCACAAC-1’ ... ]]



20 x 20 sparse Matrix of class "dgCMatrix"
                                                   
MIR1302-2HG . . . . . . . . . . . . . . . . . . . .
FAM138A     . . . . . . . . . . . . . . . . . . . .
OR4F5       . . . . . . . . . . . . . . . . . . . .
AL627309.1  . . . . . . . . . . . . . . . . . . . .
AL627309.3  . . . . . . . . . . . . . . . . . . . .
AL627309.2  . . . . . . . . . . . . . . . . . . . .
AL627309.4  . . . . . . . . . . . . . . . . . . . .
AL732372.1  . . . . . . . . . . . . . . . . . . . .
OR4F29      . . . . . . . . . . . . . . . . . . . .
AC114498.1  . . . . . . . . . . . . . . . . . . . .
OR4F16      . . . . . . . . . . . . . . . . . . . .
AL669831.2  . . . . . . . . . . . . . . . . . . . .
AL669831.5  . . 1 . . . . . . . . . . . . . . . . .
FAM87B      . . . . . . . . . . . . . . . . . . . .
LINC00115   . . 2 . . . . . . 1 . . . . . . . . . 1
FAM41C      . . . . . . . . . . . . . . . . . . . .
AL645608.7  . . . . . . . . . . . . . . . . . . . .
AL645608.3  . . . . .

In [21]:
meta <- read.csv('/lustre/scratch117/cellgen/team205/rl20/CTCL/object-new/CTCL1-8_raw_downsample0.2_anno_for_deconv_meta.csv')
head(meta)

,sample_id,donor_id,Tissue,mad_prd,ds,mad_MED,mad_MAD,mad_thr,batch,anno,broad_cell_type
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>
AAACCTGAGAATGTTG-1,CTCL_10x_1,CTCL1,Epidermis,False,0.17781403,0.07233273,0.04214057,0.240895,0,Differentiated_KC*,KC
AAACCTGAGTTCGCAT-1,CTCL_10x_1,CTCL1,Epidermis,False,0.17781403,0.07233273,0.04214057,0.240895,0,VE2,Fibro
AAACCTGCATCACAAC-1,CTCL_10x_1,CTCL1,Epidermis,False,0.19928826,0.07233273,0.04214057,0.240895,0,Differentiated_KC,KC
AAAGATGAGGAGTTTA-1,CTCL_10x_1,CTCL1,Epidermis,False,0.03918919,0.07233273,0.04214057,0.240895,0,Differentiated_KC,KC
AAAGATGGTTGTGGAG-1,CTCL_10x_1,CTCL1,Epidermis,False,0.18456376,0.07233273,0.04214057,0.240895,0,Differentiated_KC,KC
AAAGCAAAGCCTATGT-1,CTCL_10x_1,CTCL1,Epidermis,False,0.01870657,0.07233273,0.04214057,0.240895,0,Differentiated_KC,KC


In [22]:
#table(meta$broad_cell_type)
table(meta$broad_cell_type)


       APC   B/plasma      Fibro         KC     L-Endo       Mast Melanocyte 
      6652       1215       4710       7496        420        417       1187 
  Pericyte     T-cell     V-Endo 
      1837      28573       3308 

In [23]:
seurat_object@meta.data <- as.data.frame(meta)

In [24]:
#pheno.matrix <- seurat_object@meta.data[,c("donor_id","broad_cell_type","anno")]
pheno.matrix <- seurat_object@meta.data[,c("donor_id","broad_cell_type","anno")]
head(pheno.matrix)

,donor_id,broad_cell_type,anno
,<chr>,<chr>,<chr>
AAACCTGAGAATGTTG-1,CTCL1,KC,Differentiated_KC*
AAACCTGAGTTCGCAT-1,CTCL1,Fibro,VE2
AAACCTGCATCACAAC-1,CTCL1,KC,Differentiated_KC
AAAGATGAGGAGTTTA-1,CTCL1,KC,Differentiated_KC
AAAGATGGTTGTGGAG-1,CTCL1,KC,Differentiated_KC
AAAGCAAAGCCTATGT-1,CTCL1,KC,Differentiated_KC


In [25]:
metadata <- data.frame(labelDescription= c("Donor ID", "broad_cell_type", "annotation"), 
                   row.names=c("donor_id", "broad_cell_type","anno"))
metadata

,labelDescription
,<chr>
donor_id,Donor ID
broad_cell_type,broad_cell_type
anno,annotation


In [26]:
library(Matrix) 
#expr_matrix <- integrated@assays$RNA@counts #sparsematrix
gene_exprs.matrix <- gene_exprs.matrix[tabulate(summary(gene_exprs.matrix)$i) != 0, , drop = FALSE] #remove any feature without a single count
#expr_matrix <- as.matrix(gene_exprs.matrix) #convert to matrix

In [27]:
dim(gene_exprs.matrix)

[1] 27455 55815

In [28]:
expr_matrix_den <- SCOPfunctions::utils_big_as.matrix(gene_exprs.matrix, n_slices_init = 1, verbose = T) ## for large matrix

n_slices=1

converting slice 1/1

columns 1:55815

cbind dense submatrices



In [29]:
SC.eset = ExpressionSet(assayData = data.matrix(gene_exprs.matrix), 
                        phenoData =  new("AnnotatedDataFrame", data = pheno.matrix, varMetadata = metadata))
#SC.eset = ExpressionSet(assayData = expr_matrix_den, 
#                        phenoData =  new("AnnotatedDataFrame", data = pheno.matrix, varMetadata = metadata)) ## for large matrix

In [30]:
SC.eset

ExpressionSet (storageMode: lockedEnvironment)
assayData: 27455 features, 55815 samples 
  element names: exprs 
protocolData: none
phenoData
  sampleNames: AAACCTGAGAATGTTG-1 AAACCTGAGTTCGCAT-1 ...
    TTTGTCATCTTGTATC-1-40 (55815 total)
  varLabels: donor_id broad_cell_type anno
  varMetadata: labelDescription
featureData: none
experimentData: use 'experimentData(object)'
Annotation:  

In [35]:
saveRDS(SC.eset, file="/lustre/scratch117/cellgen/team205/rl20/CTCL/object-new/CTCL1-8_ExpressionSet_for_deconv_down0.2.rds")